In [1]:
# %load /home/biswadip/autoreload_jn.py
%load_ext autoreload
%autoreload 2


In [2]:
# %load sentiment\ analysis-attention.py
#!/usr/bin/env python

# In[2]:
import os

# In[49]:


import numpy as np
import pandas as pd


# In[53]:


from models.text import CustomTokenizer
from models.model import word_list, max_sentences, maxlen

In [3]:
tokenizer = CustomTokenizer(word_list = word_list)

## Load Data

In [4]:
batch_s = 32

path = "negativeReviews/"
neg_reviews = []
for f in os.listdir(path):
    file = os.path.join(path, f)
    with open(file, "r") as fl:
        neg_reviews.append(fl.read())
    

In [5]:
path = "positiveReviews//"
pos_reviews = []
for f in os.listdir(path):
    file = os.path.join(path, f)
    with open(file, "r") as fl:
        pos_reviews.append(fl.read())

In [6]:
data = pd.DataFrame(
    {"text":neg_reviews, "sentiment":0}
).append(pd.DataFrame(
    {"text":pos_reviews, "sentiment":1}
))

print("Data Shape {}".format(data.shape))
# data.to_csv("tagged_data.csv")
print("Class Distribution {}".format(
    data.sentiment.value_counts())
)

Data Shape (25000, 2)
Class Distribution 1    12500
0    12500
Name: sentiment, dtype: int64


In [7]:
data.text = data.text.apply(tokenizer.clean_text)

In [8]:
data = data.reset_index()

data = data.filter(["text","sentiment"])
data = data.sample(frac=1)

In [9]:
data = data[:5000]

In [10]:
# =================================================
import tensorflow as tf

inp = tokenizer.doc_to_sequences(data.text.tolist())

In [11]:
len(inp)

5000

## Prepare Inputs

In [12]:
inputs = []
for doc in inp:
    inputs.append(
        tf.keras.preprocessing.sequence.pad_sequences(
            doc, padding="post", value=0, maxlen=maxlen, dtype=None
        )
    )

In [13]:
a = np.zeros((len(inputs),max_sentences,maxlen))

for row,x in zip(a, inputs):
    row[:len(x)] = x[:50]

In [14]:
# Define Model

from models.model import get_model, ModelCheckpoint, max_sentences, maxlen
from models.model import HierarchicalAttentionLayer, SequenceAttentionLayer, AdditiveAttention
from models.data import Sequence_generator
# from models.tuner import tuner

In [15]:
tf

<module 'tensorflow' from '/home/biswadip/biswadip_temp/repos/bill_parsing/venv3/lib/python3.6/site-packages/tensorflow/__init__.py'>

In [16]:
model = tf.keras.models.load_model(
    "model/attention-wh_100_sh_400-10-0.87.h5",
    custom_objects={"HierarchicalAttentionLayer": HierarchicalAttentionLayer}
)

In [17]:
hirer_layer = model.layers[1]
# model.layers

sentence_layer = hirer_layer.sentence_layer
document_layer = hirer_layer.document_layer
doc_attention_layer = document_layer.attention_layer


def get_doc_attention_scores(sent):
    """Get sentences in attention weight order"""
#     sent = data.text.values[1]
    print("Original Document: \n\n {}".format(sent))
    print("\n"*3)
    sent_tokenized_index = tokenizer.doc_to_sequences([sent])
    padded_input = tf.keras.preprocessing.sequence.pad_sequences(
                sent_tokenized_index[0], padding="post", value=0, maxlen=maxlen, dtype=None
            )
    padded_input = [padded_input]

    aa = np.zeros((1,max_sentences,maxlen))

    for row,x in zip(aa, padded_input):
        row[:len(x)] = x[:max_sentences]
    print(model.predict(aa))

    sent_output = sentence_layer(aa)
    doc_lstm_hiden_states = document_layer.lstm(sent_output)
    doc_atten_scores = doc_attention_layer.get_attention_scores(
        [doc_lstm_hiden_states, doc_lstm_hiden_states]
    )

    scores = doc_atten_scores[0][0]

    sentencess = tokenizer.tokenize_sentence(sent)
    print(tf.sort(scores, direction="DESCENDING")[:len(sentencess)])
    print("\n"*3)

    print([sentencess[i] for i in tf.argsort(
        scores, direction="DESCENDING") if i<len(sentencess) ])
    print([len(sentencess[i]) for i in tf.argsort(
        scores, direction="DESCENDING") if i<len(sentencess) ])

    return [sentencess[i] for i in tf.argsort(
            scores, direction="DESCENDING") if i<len(sentencess) ]

In [18]:
hirer_layer = model.layers[1]
# model.layers

sentence_layer = hirer_layer.sentence_layer
document_layer = hirer_layer.document_layer
doc_attention_layer = document_layer.attention_layer


def get_sent_attention_scores(sent):
#     sent = data.text.values[1]
#     sent = "The Movie sucks!"
    print("Original Sentence: \n\n {}".format(sent))
    print("\n"*3)

    sent_tokenized_index = tokenizer.texts_to_sequences([sent])

    padded_input = tf.keras.preprocessing.sequence.pad_sequences(
                sent_tokenized_index, padding="post", value=0, maxlen=maxlen, dtype=None
            )

    padded_input = [padded_input]

    aa = np.zeros((1,max_sentences,maxlen))

    for row,x in zip(aa, padded_input):
        row[:len(x)] = x[:max_sentences]
    print(model.predict(aa))

    sent_output = sentence_layer(aa)

    inputs = sentence_layer.embed(aa)
    # putting every sentence in a single axis
    inputs_mask = inputs._keras_mask
    inputs = tf.reshape(
        inputs, shape = (-1 ,maxlen ,sentence_layer.embedding_len)
    )
    mask = tf.reshape(
        inputs_mask,
        shape=(-1, maxlen)
    )

    lstm_out = sentence_layer.lstm(inputs, mask=mask)

    lstm_mask = lstm_out._keras_mask
    attention_scores = sentence_layer.attention_layer.get_attention_scores(
        [lstm_out,lstm_out], 
        mask = [lstm_mask, lstm_mask]
    )
    word_indexes = tokenizer.texts_to_sequences([sent])[0]

    for i in range(len(word_indexes)):
        
        sent_attention_scores = attention_scores[i]

        first_set_attention_scores = sent_attention_scores[0]


        sorted_indexes = tf.argsort(
            first_set_attention_scores, 
            direction="DESCENDING"
        )

        ranked_words = [tokenizer.index_word[word_indexes[wx]] 
                        for wx in sorted_indexes if wx<len(word_indexes)]
        print(ranked_words)

In [19]:
from nltk import sent_tokenize

In [20]:
import re

## Sentence Ranking

In [21]:
# i=10
i+=1
text = data.text.values[i]

NameError: name 'i' is not defined

In [ ]:
text = re.sub(r"<br /><br />", " ", text).strip()

In [ ]:
sent_tokenize(text)

In [ ]:
# text = "The Movie Was awesome"
# text

In [ ]:
xxx = get_doc_attention_scores(text)
most_important_text = xxx[0]

### Most Important sentence

In [52]:
most_important_text

"Or perhaps it's the worst piece of trash ever made and it should be locked away in a vault and dumped in the Arctic Ocean."

## Word Ranking + Interaction

Word attention weights has ranking as well as interaction information. 

Attention weights has a dimension of ```(t_steps, tsteps)``` ```t_step``` is the number of words in the sentence. Normally, ```(i,j)``` element of the matrix provides how much interaction there is between ```ith and jth``` words irrespective of their distance. In general you can also take any row and the ```t_steps``` sized vector will produce the ranking of words. 

It is also observed that all the rows almost same **in case sentence attention vectors**. The attention weights in case sentences are more useful for ranking. This is expected as it is very difficult to interact sentences as more or less they provide independent information.

**In case of Word attention,** the attention weights vary across the rows(not a lot). This provides a sense that the attention weights not only create ranking but also measures the level of interctions between words to create more useful features.

In [53]:
get_sent_attention_scores(most_important_text)

Original Sentence: 

 Or perhaps it's the worst piece of trash ever made and it should be locked away in a vault and dumped in the Arctic Ocean.




[[0.71085775 0.28914222]]
['ever', 'made', 'and', 'trash', 'it', 'should', 'be', 'away', 'locked', 'vault', 'and', 'in', 'dumped', 'of', 'piece', 'a', 'in', 'the', 'ocean', 'arctic', 'worst', 'perhaps', 'or', 'unk', 'the']
['or', 'perhaps', 'unk', 'the', 'worst', 'piece', 'of', 'trash', 'ever', 'made', 'and', 'it', 'should', 'be', 'locked', 'away', 'in', 'a', 'vault', 'and', 'dumped', 'in', 'the', 'arctic', 'ocean']
['or', 'perhaps', 'unk', 'the', 'worst', 'piece', 'of', 'trash', 'ever', 'made', 'and', 'it', 'should', 'be', 'locked', 'away', 'in', 'a', 'vault', 'and', 'dumped', 'in', 'the', 'arctic', 'ocean']
['or', 'perhaps', 'unk', 'the', 'worst', 'piece', 'of', 'trash', 'ever', 'made', 'and', 'it', 'should', 'be', 'locked', 'away', 'in', 'a', 'vault', 'and', 'dumped', 'in', 'the', 'arctic', 'ocean']
['or', 'perhaps', 'unk', 'the', 'wors